# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\OscillatingDroplet3D");

## Observing sessions

In [ ]:
wmg.Sessions

In [ ]:
using System.IO;

In [ ]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [ ]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [ ]:
/*var plot = new Plot2Ddata();
var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
var time = tab["time"];
int cnt = -1;
foreach(var column in tab) {
   cnt++;
   var fmt = new PlotFormat();
   fmt.Style = Styles.Lines; 
   fmt.LineColor = allColors[cnt%allColors.Length];
   if(column.Key == "time")
      continue;
   plot.AddDataGroup(column.Key, time, column.Value, fmt);
   
   plot.Title = S1.Name;
}*/

In [ ]:
//plot.PlotNow()

In [ ]:
foreach(var s in wmg.Sessions) {
    Console.WriteLine(s);
    if(s.Timesteps.Count() > 0)
        Console.WriteLine(s.Timesteps.Last());
    else 
        Console.WriteLine("no timestep computed/saved");
    Console.WriteLine();
}

In [ ]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

## Plotting of Spherical Harmonics

In [ ]:
//string[] modes = new string[]{ "mode2", "mode3", "mode4"};
//string[] aPis = new string[] { "aP0", "aP1", "aP2" };
string[] amrS = new string[] { "arm0", "arm1" };
string[] caseS = new string[] { "case1", "case2", "case3", "case4", "case5" };

In [ ]:
Plot2Ddata[,] PlotTable = new Plot2Ddata[5,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < 5; iRow++) { // loop over cases
for(int iarm = 0; iarm < 2; iarm++) {
    string _amr = amrS[iarm];
    string _case = caseS[iRow];

    //ISessionInfo SI = null;
    ISessionInfo SI = wmg.Sessions.SingleOrDefault(sess => sess.Name.Contains(_case) 
                                                        && sess.Name.Contains(_amr));
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        var dummyPlot = new Plot2Ddata();
        dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        PlotTable[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
    var time = tab["time"];
    int cnt = -1;
    foreach(var column in tab) {
       cnt++;
       var fmt = new PlotFormat();
       fmt.Style = Styles.Lines; 
       fmt.LineColor = allColors[cnt%allColors.Length];
       if(column.Key == "time")
          continue;
       if(iarm == 0)
          fmt.DashType = DashTypes.Solid;
       else 
          fmt.DashType = DashTypes.Dashed;
       string name = column.Key;
       if(iarm > 0)
           name = name + "-amr";
       plot.AddDataGroup(name, time, column.Value, fmt);
   }
   
   plot.ShowLegend = iCol == 2 && iRow == 2;
   
   //plot.Title = SI.Name;
   
   if(iarm == 0) {
       PlotTable[iRow,iCol] = plot;
   } else {
       PlotTable[iRow,iCol] = PlotTable[iRow,iCol].Merge(plot);
   }
   
}

if(PlotTable[iRow,iCol] != null) {
    PlotTable[iRow,iCol].ShowLegend = iCol == 0 && iRow == 0;
    //if(iRow == 0)
    //   PlotTable[iRow,iCol].Title = aPis[iCol];
    if(iCol == 0)
       PlotTable[iRow,iCol].Ylabel = caseS[iRow];
}
}
}


In [ ]:
var gp = PlotTable.ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

In [ ]:
var cl = gp.PlotCairolatex(xSize:32,ySize:18);
cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}